In [ ]:
import sys 
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import pickle
from utils import data_augmentation
from tensorflow import keras
from sklearn.metrics import classification_report
from utils import utils

font = {
    'family' : 'normal',
    'weight': 'normal',
    'size': 18
}
matplotlib.rc('font', **font)

In [ ]:
# Selecting the GPU to be used 
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    # Restrict tensor flow to use GPU-1
    try:
        tf.config.experimental.set_visible_devices([], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Set GPUs before initializing
        print(e)

# Multi-output model evaluation and Plotting

## Load data without oversampling

In [ ]:
# Directory of the dataset
data_dir = os.path.join(os.getcwd(), "model_data", "multi_output_axisfr")

for index1, file in enumerate(os.listdir(data_dir)):
    data = np.load(os.path.join(data_dir, file), allow_pickle=True)[()]
    
    for index2, ((axis, feed_rate), segmented_points) in enumerate(data.items()):
        
        temp_axis = np.repeat(axis, segmented_points.shape[0])[:, np.newaxis]
        temp_fr = np.repeat(feed_rate, segmented_points.shape[0])[:, np.newaxis]
    
        # part of X and y
        if index2 == 0:
            # part of y for an axis and all feed rate
            part_y_axis = temp_axis
            part_y_fr = temp_fr
            # part of X
            part_X = segmented_points
        else:
            part_y_axis = np.append(part_y_axis, temp_axis, axis=0)
            part_y_fr = np.append(part_y_fr, temp_fr, axis=0)
            part_X = np.append(part_X, segmented_points, axis=0)
            
    if index1 == 0:
        # y
        y_axis = part_y_axis
        y_fr = part_y_fr
        # X
        X = part_X
    else:
        y_axis = np.append(y_axis, part_y_axis, axis=0)
        y_fr = np.append(y_fr, part_y_fr, axis=0)
        X = np.append(X, part_X, axis=0)



## Recovering the training history and scores for verification

- To get an idea on the training performance

In [ ]:
# The folder time - determine
folder_time = "2022-06-22T22:15:53.705105"

In [ ]:
# Select item to load
history_path = os.path.join(os.getcwd(), "model_weights", "multi_output_ax-fr", folder_time, "training_history", "history.pickle")
score_path = os.path.join(os.getcwd(), "model_weights", "multi_output_ax-fr", folder_time, "training_history", "score.pickle")

with open(history_path, "rb") as fh:
    history = pickle.load(fh)
    
with open(score_path, "rb") as fh:
    score = pickle.load(fh)

In [ ]:
print("=============================== Accuracies ==============================")
print(f"Maximum axis-detection training accuracy {max(history[0]['axis_detection_accuracy'])}")
print(f"Maximum axis-detection validation accuracy {max(history[0]['val_axis_detection_accuracy'])}")

print("=============================== Losses ==================================")
print(f"Minimum axis-detection training loss {min(history[0]['axis_detection_loss'])}")
print(f"Minimum axis-detection validation loss {min(history[0]['val_axis_detection_loss'])}")

In [ ]:
# Find the index of maximum
val_losses = []
for kfold in history.keys():
    val_losses.append(min(history[kfold]["val_loss"]))
sys.stdout.write(f"The validation losses for each of the fold is given by {val_losses}\n")
sys.stdout.write(f"The K-fold with the best performance is {val_losses.index(min(val_losses))}\n")
# Select the fold
selected_fold = val_losses.index(min(val_losses))

In [ ]:
print("The scores are given below")
print(score[selected_fold])

## Plotting model evaluation results

- Classification report
- F1-Score and associated metrics
- Error Histogram
- Axis Detection confusion matrix
- Feed rate prediction results
- Evaluation of the CV fold performances

In [ ]:
# Select the model to load
load_file = f"multi-output_KFold-{selected_fold}_model.h5"
# Load the selected model 
model = keras.models.load_model(os.path.join(os.getcwd(), "model_weights", "multi_output_ax-fr", folder_time, load_file))
# Need compilation to change the accuracy metric - tensorflow issue
adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss={"axis_detection": "sparse_categorical_crossentropy", "feed_rate_prediction": "mse"}, 
              metrics={"axis_detection": "accuracy"}, loss_weights=[1, 20])

model.evaluate(X, {"axis_detection": y_axis, "feed_rate_prediction": y_fr})

In [ ]:
# Predict using the model 
y_pred = model.predict(X)
# Predicting the axis
y_pred_axis = np.argmax(y_pred[0], axis=1)
# Feed rates
y_pred_fr = y_pred[1]

### Classification Report

In [ ]:
# Classification Report
target_classes = ["X", "Y", "Z", "B", "C"]

print("Classification Report")
print(classification_report(y_axis, y_pred_axis, target_names=target_classes))


### AUC Curve

In [ ]:
# The AUC score 
from sklearn.metrics import roc_auc_score

# Getting the probability scores
y_score = y_pred[0]
classes = [0, 1, 2, 3, 4]

# Get the score
roc_auc = roc_auc_score(y_axis, y_score, average="macro", multi_class="ovr", labels=classes)

print(f'The AOC value is {roc_auc}')

### R2 Score for regression

In [ ]:
# Regression report
# Computation for R^2 and RMSE
from sklearn.metrics import r2_score

r2 = r2_score(y_fr, y_pred_fr)

print(f"The R^2 value for regression is {r2}")

### Error histogram

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0, 0, 1, 1])

# error histogram
error = y_fr - y_pred_fr

std = np.std(error)
print(f"The standard deviation is given by {std}")

hist = ax.hist(error, bins=50, range=(-84, +84), histtype="bar", color="red")

ax.set_title("Error histogram")
ax.set_xlabel(r"Error $(y_i - \hat{y}_i)$")
ax.set_ylabel("Frequency")
#fig.savefig("error_history.png", bbox_inches="tight")

### Error histogram by axis


In [ ]:
# Get the figures
fig = plt.figure(figsize=(30, 20))
ax = fig.subplots(nrows=2, ncols=3)

# Create a dataframe of all Ys
y_df = pd.DataFrame(data=np.concatenate((np.expand_dims(y_pred_axis, axis=1), y_pred_fr, y_fr), axis=1), columns=["pred_yaxis", "pred_yfr", "y_fr"], dtype="float32", copy=True)
y_df["fr_error"] = y_df["y_fr"] - y_df["pred_yfr"]
y_df_group = y_df.groupby(by="pred_yaxis")

axes_conversion = ["X", "Y", "Z", "B", "C"]
row = 0
col = 0
for index, axis_index in enumerate(y_df_group.groups.keys()):
    # Get the error values
    error = y_df_group.get_group(axis_index)["fr_error"]

    # Normalize the standard deviation if required

    # Plot the histogram
    hist = ax[row, col].hist(error, bins=50, range=(-100, +100), histtype="bar", color="red")
    # Set the title appropriately
    ax[row, col].set_title(f"Error histogram for {axes_conversion[int(axis_index)]}-axis with $\sigma$ = {np.std(error.to_numpy()).round(decimals=0)}")
    ax[row, col].set_xlabel(r"Error $(y_i - \hat{y}_i)$")
    ax[row, col].set_ylabel("Frequency")

    # Handle row and col conversions
    col += 1
    if col > 2:
        row += 1
        col = 0

# Delete the un-plotted axis
plt.delaxes(ax[-1, -1])



### Confusion matrix

In [ ]:
# Determining and plotting the confusion matrix
class_names = ["X-axis", "Y-axis", "Z-axis", "B-axis", "C-axis"]
con_mat = tf.math.confusion_matrix(labels=y_axis, predictions=y_pred_axis).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1) [:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm, index=class_names, columns=class_names)
figure = plt.figure(figsize=(10,8))
sns.set(font_scale = 2)
sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel("True label")
plt.xlabel("Predicted label")

#plt.savefig("conmat-temp.png", bbox_inches="tight")

### Feed rate prediction

In [ ]:
# Extract the predicted feed rates 
results_df = pd.DataFrame({"axis_act": np.squeeze(y_axis), "fr_act": np.squeeze(y_fr), "axis_pred": np.squeeze(y_pred_axis), "fr_pred": np.squeeze(y_pred_fr)})
temp = results_df.groupby(by=["axis_act", "fr_act"])

plot_data = {}
checked_list = []
for index, group in enumerate(temp.groups.keys()):
    
    if group[0] not in checked_list:
        temp2 = temp.get_group(group)
        temp2 = temp2[temp2["axis_act"] == temp2["axis_pred"]]
        plot_data[group[0]] = np.array([[group[1], temp2["fr_pred"].mean()]])
        checked_list.append(group[0])
    else:
        temp2 = temp.get_group(group)
        temp2 = temp2[temp2["axis_act"] == temp2["axis_pred"]]
        plot_data[group[0]] = np.append(plot_data[group[0]], np.array([[group[1], temp2["fr_pred"].mean()]]), axis=0)

In [ ]:
# Set the font properties
matplotlib.rcdefaults()
font = {'size'   : 5.0}
matplotlib.rc('font', **font)

# PLotting of the feed rate prediction
fig, axs = plt.subplots(5, 1, sharey=False, figsize=(20, 15))

for axis in sorted(plot_data.keys()):
    
    # Get the actual and predicted feed
    actual_feed = plot_data[axis][:, 0]
    predicted_feed = plot_data[axis][:, 1]
    labels = [str(round(x)) for x in actual_feed]
    
    # PLotting the bar chart
    x = np.arange(len(labels)) # the label locations
    width = 0.35 # the width of the bars 
    
    rects1 = axs[axis].bar(x - width/2, actual_feed, width, label="Actual")
    rects2 = axs[axis].bar(x + width/2, predicted_feed, width, label="Predicted")
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    axs[axis].set_xticks(x)
    axs[axis].set_xticklabels(labels)
    axs[axis].legend(loc="upper left")
    
    #fig.savefig("feed_rate_pred.png")

## CV fold evaluation

In [ ]:
score_load_loc = os.path.join(os.getcwd(), "model_weights", "multi_output_ax-fr", folder_time, "training_history", "score.pickle")

# Load the data
with open(score_load_loc, "rb") as fhandle:
    scores = pickle.load(fhandle)


In [ ]:
df = pd.DataFrame(scores, index=["cummulative_loss", "axis_detection_loss", "feed_rate_prediction_loss", "axis_detection_accuracy"])
df = df.transpose()

In [ ]:
sys.stdout.write("Feed rate prediction MSE\n")
df.describe()

In [ ]:
sys.stdout.write("Feed rate prediction RMSE\n")
df["feed_rate_prediction_loss"] = np.sqrt(df["feed_rate_prediction_loss"])
df.describe()

In [ ]:
sys.stdout.write("Feed rate prediction loss\n")
df[["feed_rate_prediction_loss"]].boxplot()

In [ ]:
sys.stdout.write("Axis detection loss\n")
df[["axis_detection_loss"]].boxplot()

In [ ]:
sys.stdout.write("Axis detection loss\n")
df[["axis_detection_accuracy"]].boxplot()